<a href="https://colab.research.google.com/github/David-Cunha/distilbert-base-uncased-finetuned-financial-news-sentiment/blob/main/Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instala as bibliotecas para manipulação de dados, acesso ao Hugging Face,
# modelos de transformers, criação de datasets e busca vetorial.
!pip install -q \
    kaggle \
    pandas \
    transformers[torch] \
    datasets \
    faiss-cpu \
    sentence-transformers \
    huggingface_hub \
    accelerate \
    evaluate

In [ ]:
!pip install -q --upgrade transformers>=4.35.0
!pip install -q --upgrade accelerate

In [ ]:
# @title Configuração das Variáveis de Ambiente e Autenticação
import os
from google.colab import userdata

# Configura as variáveis de ambiente para a API do Kaggle
# Isso permite que a biblioteca do Kaggle autentique e baixe datasets.
os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')

# Recupera o token do Hugging Face para login posterior
# Este token permitirá o upload do nosso modelo fine-tunado.
HUGGINGFACE_TOKEN = userdata.get('HUGGINGFACE_HUB_TOKEN')

print("Variáveis de ambiente configuradas com sucesso!")

In [ ]:
# @title Download e extração do dataset para Fine-Tuning
# Dataset: Sentiment Analysis for Financial News (https://www.kaggle.com/datasets/ankurzing/sentiment-analysis-for-financial-news)
!kaggle datasets download -d ankurzing/sentiment-analysis-for-financial-news -p ./data/
!unzip -q -o ./data/sentiment-analysis-for-financial-news.zip -d ./data/
print("Dataset de notícias financeiras baixado e extraído com sucesso!")

In [ ]:
# @title Carregamento e pré-processamento dos dados
import faiss
import numpy as np
import pandas as pd
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

# Carrega o dataset e renomeia as colunas para um formato mais padrão
df_sentiment = pd.read_csv(
    './data/all-data.csv',
    encoding='latin-1',
    header=None,
    names=['sentiment', 'text']
)

# Mapeia os sentimentos (strings) para IDs numéricos (inteiros)
labels_map = {'neutral': 0, 'positive': 1, 'negative': 2}
df_sentiment['label'] = df_sentiment['sentiment'].map(labels_map)

# Remove a coluna de texto original
df_sentiment = df_sentiment.drop(columns=['sentiment'])

# Converte o DataFrame do Pandas para um objeto Dataset da biblioteca `datasets`
dataset = Dataset.from_pandas(df_sentiment)

# Divide o dataset em 80% para treino e 20% para teste
train_test_split = dataset.train_test_split(test_size=0.2, shuffle=True, seed=42)
dataset_dict = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})

# Carrega o tokenizador do modelo que vamos usar
# 'distilbert-base-uncased' é um modelo leve e eficaz, ótimo para fine-tuning rápido.
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Função para tokenizar o texto
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# Aplica a tokenização a todo o dataset de forma eficiente
tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

print("Estrutura do dataset tokenizado:")
print(tokenized_datasets)

In [ ]:
# @title Treinamento do modelo
import numpy as np
import evaluate
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# Carrega o modelo pré-treinado, especificando o número de rótulos da nossa tarefa
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(labels_map),
    id2label={v: k for k, v in labels_map.items()}, # Mapeamento reverso para visualização
    label2id=labels_map
)

# Define os argumentos de treinamento
# Estes são os hiperparâmetros do nosso treinamento
training_args = TrainingArguments(
    output_dir="distilbert-sentiment-finetuned",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2, # Apenas 2 épocas para um treinamento rápido
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="none",
)

# Define a métrica de avaliação (acurácia)
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Cria a instância do Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Inicia o treinamento
print("Iniciando o fine-tuning...")
trainer.train()
print("Treinamento concluído!")

In [ ]:
# @title Avaliação do modelo no conjunto de teste
eval_results = trainer.evaluate()

print("\nResultados da avaliação no conjunto de teste:")
print(f"Acurácia: {eval_results['eval_accuracy']:.4f}")
print(f"Loss: {eval_results['eval_loss']:.4f}")

In [ ]:
# @title Login no Hugging Face Hub e publicação do modelo
from huggingface_hub import HfApi, HfFolder, login

# Faz o login programaticamente usando o token salvo
login(token=HUGGINGFACE_TOKEN)

# Define o nome do repositório no Hub
# É uma boa prática incluir seu nome de usuário para evitar conflitos
hub_username = HfApi().whoami()['name']
repo_name = f"{hub_username}/distilbert-finetuned-sentiment-financial-news"

print(f"Preparando para enviar o modelo para: {repo_name}")

# Envia o modelo e o tokenizador para o Hub
# A função `push_to_hub` cria o repositório se ele não existir
trainer.push_to_hub(repo_name, commit_message="Primeiro commit do modelo treinado")

print("\nModelo enviado com sucesso para o Hugging Face Hub!")
print(f"Você pode encontrá-lo em: https://huggingface.co/{repo_name}")